# Programming LLMs with Language

In previous chapters, we saw how we can use the semantic information in large language models (LLMs) through standard nlp pipelines (e.g., classification) or using their underlying embeddings. But LLMs are *natively* programmed through language. Moreover, recent scholarship has both [**argued**](https://aclanthology.org/2023.findings-acl.247/) and then [**demonstrated**](https://proceedings.mlr.press/v202/von-oswald23a.html) how LLMs perform gradient descent on their internal, high-dimensional representations to optimize a response to a language prompt.

In this notebook accompanying the chapter on programming LLMs with language, we will explore various ways we can interact with some of the most powerful deep learning models today using language.

# Set Up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Imports

In [ ]:
import os
import difflib
import re

# for data manipulation
import pandas as pd
import numpy as np

# for OpenAI
! pip install openai==1.10.0

import openai
from openai import OpenAI
import getpass

# for NLP tasks
import nltk
import spacy
from spacy import displacy
try:
    nlp = spacy.load("en")
except OSError:
    nlp = spacy.load("en_core_web_sm")

In [ ]:
# declare api key
# %env OPENAI_API_KEY =

In [4]:
client = OpenAI()

## Helper Functions

In [5]:
# get random indices
def random_indices(df_len, num_indices):
    """
    Generate a list of random indices.

    returns: list of random indices
    """
    random_indices = np.random.choice(df_len, num_indices, replace=False)
    random_indices = list(random_indices)
    random_indices.sort()

    return random_indices

## Data Loading

In [6]:
scotus_fp = '/content/drive/MyDrive/Colab Notebooks/soci40133-homeworks/data/scotus_cases_cleaned.csv'
scotus_df = pd.read_csv(scotus_fp)
scotus_df.head()

,case,year,author,pdf_url,raw_text,cleaned_text
0,Dobbs v. Jackson Women's Health Organization,2022,"Samuel A. Alito, Jr.",https://www.supremecourt.gov/opinions/21pdf/19...,\n \n \n \n \n \n \n \n \n \n \n ...,"1 (Slip Opinion) OCTOBER TERM, 2021 Syllabus N..."
1,Whole Woman's Health v. Hellerstedt,2016,Stephen Breyer,https://supreme.justia.com/cases/federal/us/57...,\n \n \n \n \n \n \n \n \n \n \n \n \...,"1 (Slip Opinion) OCTOBER TERM, 2015 Syllabus N..."
2,Gonzales v. Carhart,2007,Anthony Kennedy,https://tile.loc.gov/storage-services/service/...,550US1\nU\nnit:\n$U31\n[07-28-10\n12:14:15]\nP...,550US1 U nit: U31 07 28 10 12:14:15 P A GES PG...
3,Stenberg v. Carhart,2000,Stephen Breyer,https://tile.loc.gov/storage-services/service/...,"OCTOBER TERM, 1999\nSyllabus\nSTENBERG, ATTORN...","OCTOBER TERM, 1999 Syllabus STENBERG, ATTORNEY..."
4,Planned Parenthood of Southeastern Pennsylvani...,1992,"Anthony Kennedy, David Souter, Sandra Day O’Co...",https://supreme.justia.com/cases/federal/us/50...,505us3u117 07-09-96 09:34:02 PAGES OPINPGT\n83...,505us3u117 07 09 96 09:34:02 PAGES OPINPGT 833...


In [7]:
congress_fp = '/content/drive/MyDrive/Colab Notebooks/soci40133-homeworks/data/congress_legislation_cleaned.csv'
congress_df = pd.read_csv(congress_fp)
congress_df.head()

,legislation number,url,congress,title,date proposed,amends amendment,latest summary,congress_num,bill_type,bill_num,api_url,text_url,raw_text,cleaned_text,cleaned_summary
0,H.R. 2907,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Let Doctors Provide Reproductive Health Care Act,NaN,NaN,<p><strong>Let Doctors Provide Reproductive H...,118,hr,2907,https://api.congress.gov/v3/bill/118/hr/2907/t...,https://www.congress.gov/118/bills/hr2907/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Let Doctors Provide Reproductive Health Care A...
1,S. 1297,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),Let Doctors Provide Reproductive Health Care Act,NaN,NaN,<p><strong>Let Doctors Provide Reproductive H...,118,s,1297,https://api.congress.gov/v3/bill/118/s/1297/te...,https://www.congress.gov/118/bills/s1297/BILLS...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Let Doctors Provide Reproductive Health Care A...
2,H.R. 4901,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Reproductive Health Care Accessibility Act,NaN,NaN,<p><strong>Reproductive Health Care Accessibi...,118,hr,4901,https://api.congress.gov/v3/bill/118/hr/4901/t...,https://www.congress.gov/118/bills/hr4901/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Reproductive Health Care Accessibility Act Thi...
3,S. 2544,https://www.congress.gov/bill/118th-congress/s...,118th Congress (2023-2024),Reproductive Health Care Accessibility Act,NaN,NaN,<p><strong>Reproductive Health Care Accessibi...,118,s,2544,https://api.congress.gov/v3/bill/118/s/2544/te...,https://www.congress.gov/118/bills/s2544/BILLS...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,Reproductive Health Care Accessibility Act Thi...
4,H.R. 4147,https://www.congress.gov/bill/118th-congress/h...,118th Congress (2023-2024),Reproductive Health Care Training Act of 2023,NaN,NaN,NaN,118,hr,4147,https://api.congress.gov/v3/bill/118/hr/4147/t...,https://www.congress.gov/118/bills/hr4147/BILL...,\n[Congressional Bills 118th Congress]\n[From ...,Congressional Bills 118th Congress From the U....,None


In [ ]:
congress_df.loc[[20,40,60,80,100,120,140], ['legislation number', 'congress', 'title']]

,legislation number,congress,title
20,H.R. 1470,118th Congress (2023-2024),No Federal Funds for Abortion Travel Expenses ...
40,S. 2089,118th Congress (2023-2024),"A bill to amend title 40, United States Code, ..."
60,S. 2439,118th Congress (2023-2024),Reproductive Health Patient Navigator Act of 2023
80,H.R. 1771,118th Congress (2023-2024),Pregnancy Resource Center Defense Act
100,H.R. 128,118th Congress (2023-2024),Defund Planned Parenthood Act of 2023
120,H.R. 9040,117th Congress (2021-2022),Reproductive Health Care Accessibility Act
140,H.R. 243,117th Congress (2021-2022),Title X Abortion Provider Prohibition Act


#### Uploading raw and cleaned text

In [ ]:
dobbs_syllabus_raw_fp = '/content/drive/MyDrive/Colab Notebooks/soci40133-homeworks/data/dobbs_syllabus_raw.txt'
dobbs_syllabus_cleaned_fp = '/content/drive/MyDrive/Colab Notebooks/soci40133-homeworks/data/dobbs_syllabus_cleaned'
with open(dobbs_syllabus_raw_fp, 'r') as file:
  dobbs_syllabus_raw = file.read()

with open(dobbs_syllabus_cleaned_fp, 'r') as file:
  dobbs_syllabus_cleaned = file.read()

In [ ]:
dobbs_syllabus_raw

'  \n \n \n  \n    \n       \n \n  \n \n \n \n \n \n    \n  \n \n \n  \n   \n \n \n \n \n \n \n  \n 1 (Slip Opinion) OCTOBER TERM, 2021 \nSyllabus \nNOTE: Where it is feasible, a syllabus (headnote) will be released, as is \nbeing done in connection with this case, at the time the opinion is issued. \nThe syllabus constitutes no part of the opinion of the Court but has been prepared by the Reporter of Decisions for the convenience of the reader. See United States  v. Detroit Timber & Lumber Co.,  200 U. S. 321, 337. \nSUPREME COURT OF THE UNITED STATES \nSyllabus \nDOBBS, STATE HEALTH OFFICER OF THE \nMISSISSIPPI DEPARTMENT OF HEALTH, ET AL . v. \nJACKSON WOMEN’S HEALTH ORGANIZATION ET AL . \nCERTIORARI TO THE UNITED STATES COURT OF APPEALS FOR \nTHE FIFTH CIRCUIT \nNo. 19–1392. Argued December 1, 2021—Decided June 24, 2022 \nMississippi’s Gestational Age Act prov ides that “[e]xcept in a medical\nemergency or in the case of a seve re fetal abnormality, a person shall\nnot intentionall

#### Truncate example

In [ ]:
def replace_func(match):
    # Remove spaces around the dash
    return re.sub(r'\s*[-–]\s*', '–', match.group(0))

In [ ]:
dobbs_trunc_syllabus = dobbs_syllabus_cleaned[dobbs_syllabus_cleaned.index('Mississippi'):]
dobbs_trunc_syllabus = re.sub(r'Pp\. \d+\s*[-–]\s*\d+\.', '', dobbs_trunc_syllabus)
dobbs_trunc_syllabus = ' '.join([i.strip() for i in dobbs_trunc_syllabus.split() if len(i.strip()) > 0])
dobbs_trunc_syllabus = re.sub('Syllabus', '', dobbs_trunc_syllabus).replace('T HOMAS', 'THOMAS').replace('R OBERTS', 'ROBERTS').replace('K AVANAUGH', 'KAVANAUGH').replace('B ARRETT', 'BARRETT').replace('B REYER', 'BREYER').replace('K AGAN', 'KAGAN').replace('U. S.', 'U.S.')
# Replace the matches in dobbs_trunc_syllabus using the replace_func for each match
dobbs_trunc_syllabus=re.sub(r'\b\d+\s*[-–]\s*\d+\b', replace_func, dobbs_trunc_syllabus)
dobbs_trunc_syllabus=re.sub(r'\([A-Za-z\.\,\s]+\)\.', '', dobbs_trunc_syllabus)
dobbs_trunc_syllabus = re.sub(r'\s+\d+ DOBBS v. JACKSON WOMEN\’S HEALTH ORGANIZATION ', '', dobbs_trunc_syllabus).replace('20t h', '20th')
dobbs_trunc_syllabus=re.sub(r'\s+\d Cite as\: 597 U\.S\. ____ \(2022\)  ', '', dobbs_trunc_syllabus)

## <font color="blue">*Exercise 1*</font>

<font color="blue">As this week's challeging questions asks, we'd like you to think how LLM can help your final project.  Try to use the OpenAI API to analyze a small-sized dataset (Remember to monitor API use on your OpenAI account!). The data could a sample from the dataset you prepare for the final project or some others. If it's going be a conventional task like classification, compare and see how it could beat(or being defeated) by other algorithms you've learned from previous weeks. If it's a speical task that you cannot find a learned algorithm to compare with, evaluate its performance on your own and try if you can improve by changing hyperparameters(see [here](https://platform.openai.com/docs/api-reference/chat/create)), the prompt, etc.

### Correcting extracted text
There are a lot of errors in the extracted legislation, where the spaces between words will have been removed or words, like "human", will have been split of erroneously to be something like "hu man", all of which affect tokenization.

There isn't a specific model that I can compare this to (there are other LLMs, but that defeats the purpose of this assignment), so I will manually edit the syllabus of the Dobbs v. Jackson legislation and compare it to ChatGPT's approach.

#### Buidling model

#### Running it against an example

In [ ]:
# running an example
prompt = {"role": "system", "content": """
    You are a legislation-cleaning assistant. You correct legislative text delimited with triple quotes by
    identifying and fixing errors such as misplaced or duplicate spaces, missing spaces,
    removal of new line characters, and identification and concatenation of split up words.

    Example of incorrect text: \"\"\"\n\n\nThis is a pi ece of legis-lation that   nee ds to\n\n becleaned..\"\"\"'

    Example of corrected text: \"\"\"This is a piece of legislation that needs to be cleaned.\"\"\"
    """}

example = {"role": "user",
           "content": """Please clean this piece of text:
           \"\"\"
           The quickbro wn \nfoxjump sover thelazy dog without a care in the wor-ld.
           \"\"\"
           """}

# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[prompt, example],
)

In [ ]:
response.choices[0].message.content

'"""The quick brown fox jumps over the lazy dog without a care in the world."""'

#### Running it against the actual legislation

In [ ]:
# split the syllabus for easier model parsing
syllabus_len = len(dobbs_syllabus_raw)

In [ ]:
split1 = syllabus_len//3
split2 = split1+(syllabus_len//3-2)

In [ ]:
dobbs_syllabus_raw1 = dobbs_syllabus_raw[:split1]
dobbs_syllabus_raw2 = dobbs_syllabus_raw[split1:split2]
dobbs_syllabus_raw3 = dobbs_syllabus_raw[split2:]

In [ ]:
# Continuing the conversation with a new piece of text
dobbs_clean_message1 = {
    "role": "user",
    "content": f"Please clean this new piece of text: \"\"\"{dobbs_syllabus_raw1}\"\"\""
}

dobbs_clean_message2 = {
    "role": "user",
    "content": f"Please clean this new piece of text: \"\"\"{dobbs_syllabus_raw2}\"\"\""
}

dobbs_clean_message3 = {
    "role": "user",
    "content": f"Please clean this new piece of text: \"\"\"{dobbs_syllabus_raw3}\"\"\""
}

# Append the new message to the existing conversation
messages1 = [messages[0]] + [dobbs_clean_message1]
messages2 = [messages[0]] + [dobbs_clean_message2]
messages3 = [messages[0]] + [dobbs_clean_message3]

In [ ]:
response1 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages1,
)

In [ ]:
response2 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages2,
)

In [ ]:
response3 = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages3,
)

In [ ]:
full_response = ''

for resp in [response1, response2, response3]:
  text = resp.choices[0].message.content.split("\n\n")[-1].strip('"') + ' '
  full_response += text

print(full_response)


1 (Slip Opinion) OCTOBER TERM, 2021
Syllabus
NOTE: Where it is feasible, a syllabus (headnote) will be released, as is being done in connection with this case, at the time the opinion is issued. The syllabus constitutes no part of the opinion of the Court but has been prepared by the Reporter of Decisions for the convenience of the reader. See United States v. Detroit Timber & Lumber Co., 200 U. S. 321, 337.
SUPREME COURT OF THE UNITED STATES
Syllabus
DOBBS, STATE HEALTH OFFICER OF THE MISSISSIPPI DEPARTMENT OF HEALTH, ET AL. v. JACKSON WOMEN’S HEALTH ORGANIZATION ET AL.
CERTIORARI TO THE UNITED STATES COURT OF APPEALS FOR THE FIFTH CIRCUIT
No. 19–1392. Argued December 1, 2021—Decided June 24, 2022
Mississippi’s Gestational Age Act provides that “[e]xcept in a medical emergency or in the case of a severe fetal abnormality, a person shall not intentionally or knowingly perform . . . or induce an abortion of an unborn human being if the probable gestational age of the unborn human being

#### Removing formatting
The GPT model was able to retain new-line formatting that show legislation sections, which is an unexpected perk that I did not do in my manually cleaned version.

In [ ]:
full_response = full_response.replace('\n', ' ').strip(' ')

#### Comparison

In [ ]:
# Instantiate Differ object
d = difflib.Differ()

# Compare the two texts
diff = list(d.compare(full_response.splitlines(), dobbs_syllabus_cleaned.splitlines()))

# Print the differences
print('\n'.join(diff))

- 1 (Slip Opinion) OCTOBER TERM, 2021 Syllabus NOTE: Where it is feasible, a syllabus (headnote) will be released, as is being done in connection with this case, at the time the opinion is issued. The syllabus constitutes no part of the opinion of the Court but has been prepared by the Reporter of Decisions for the convenience of the reader. See United States v. Detroit Timber & Lumber Co., 200 U. S. 321, 337. SUPREME COURT OF THE UNITED STATES Syllabus DOBBS, STATE HEALTH OFFICER OF THE MISSISSIPPI DEPARTMENT OF HEALTH, ET AL. v. JACKSON WOMEN’S HEALTH ORGANIZATION ET AL. CERTIORARI TO THE UNITED STATES COURT OF APPEALS FOR THE FIFTH CIRCUIT No. 19–1392. Argued December 1, 2021—Decided June 24, 2022 Mississippi’s Gestational Age Act provides that “[e]xcept in a medical emergency or in the case of a severe fetal abnormality, a person shall not intentionally or knowingly perform . . . or induce an abortion of an unborn human being if the probable gestational age of the unborn human bein

#### Reflection
Through a comparison of how well the OpenAI model did in comparison to me when cleaning the legislation text, the model proved to actually do better than I did at preserving grammatical conventions, while also displaying a similar ability to my own at identifying words that were erroneously split and removing extra white spaces and new line characters.

## <font color="blue">*Exercise 2*</font>

<font color="blue">Fine-tune an LLM. You can either use the model (llama-2-7b) in the example code or find another open-source LLM. You may use datasets provided by HuggingFace or a dataset you collect from somewhere else (for your final project). If the task happens to be the same as in exercise 1, You can choose to compare the performance between the OpenAI LLM and your fine-tuned LLM. You can also choose to compare the performance between the vanilla and the fine-tuned LLM.

### Identifying Named-Entities
Specifically with this task, I want to identify references to other legislation within a SCOTUS decision, as well as key individuals. With this information, I want to be able to provide brief descriptions of each legislation and individual.

*Note: I did not want to purchase compute units in addition to the purchases I made to access the OpenAI API, so I will be comparing a vanilla OpenAI model to a fine-tuned one.*

#### Vanilla Model Building & Example

In [ ]:
text = """
Mississippi’s Gestational Age Act provides that “[e]xcept in a medical emergency or in the case of a severe fetal abnormality, a person shall not intentionally or knowingly perform... or induce an abortion of an unborn human being if the probable gestational age of the unborn human being has been determined to be greater than fifteen (15) weeks.” Miss. Code Ann. §41–41–191. Respondents—Jackson Women’s Health Organization, an abortion clinic, and one of its doctors—challenged the Act in Federal District Court, alleging that it violated this Court’s precedents establishing a constitutional right to abortion, in particular Roe v. Wade, 410 U.S. 113, and Planned Parenthood of Southeastern Pa. v. Casey, 505 U.S. 833. The District Court granted summary judgment in favor of respondents and permanently enjoined enforcement of the Act, reasoning that Mississippi’s 15-week restriction on abortion violates this Court’s cases forbidding States to ban abortion pre-viability. The Fifth Circuit affirmed. Before this Court, petitioners defend the Act on the grounds that Roe and Casey were wrongly decided and that the Act is constitutional because it satisfies rational-basis review. Held: The Constitution does not confer a right to abortion; Roe and Casey are overruled; and the authority to regulate abortion is returned to the people and their elected representatives.
"""


In [ ]:
# running an example
prompt ={"role": "system", "content": """
    You are a named-entity recognition assistant, specifically for United States Supreme Court legislation.
    Your job is to identify key pieces of legislation, court cases, and individuals referenced within a Supreme Court opinion, where the opinion is delimited by triple quotes.
    Once these entities are recognized, you are to provide the frequency with which the entity is refrenced throughout the text
    and provide a 1-2 sentence summary of the identified entity.

    Example request: Please pull the legislative entities from this SCOTUS opinion and provide brief descriptions of the top 5 most frequent entities:
    \"\"\"
    Respondents—Jackson Women’s Health Organization, an abortion clinic, and one of its doctors—challenged the Act in Federal District Court, alleging that it violated this Court’s precedents establishing a constitutional right to abortion, in particular Roe v. Wade, 410 U. S. 113, and Planned Parenthood of Southeastern Pa. v. Casey, 505 U. S. 833.
    \"\"\"

    Example response:
    \"\"\"
    Mississippi Gestational Act (frequency=1): Also referred to as 'the Act.' A ruling that prohibits abortions after a fetus's gestational age is more than 15 weeks. The law also has narrow exceptions for medical emergencies or "a severe fetal abnormality".
    Federal District Court (frequency=1): The district courts are the general trial courts of the federal court system. Each district court has at least one United States District Judge, appointed by the President and confirmed by the Senate for a life term.
    The Supreme Court of the United States (frequency=1): Also referred to as "this court". The highest court in the federal judiciary of the United States. It has ultimate appellate jurisdiction over all U.S. federal court cases, and over state court cases that turn on questions of U.S. constitutional or federal law.
    Roe v. Wade, 410 U. S. 113 (frequency=1): A landmark Supreme Court decision that ruled that the Constitution protects a woman's right to an abortion. The court's decision recognized that the right to liberty in the Constitution includes the right to decide whether to continue a pregnancy.
    Planned Parenthood of Southeastern Pa. v. Casey, 505 U. S. 833 (frequency=1): A landmark Supreme Court case that reaffirmed the right to an abortion established by Roe v. Wade (1973). The case established that a person has the right to an abortion, but the state can ban abortions of viable fetuses except when the mother's health is at risk.
    \"\"\"
    """}
example = {"role": "user", "content": """
          Please pull the legislative entities, frequencies, and descriptions from this extract of a SCOTUS opinion:
          \"\"\"
          The Casey Court grounded its decision solely on the theory that the right to obtain an abortion is part of the “liberty” protected by the Fourteenth Amendment’s Due Process Clause. Others have suggested that support can be found in the Fourteenth Amendment’s Equal Protection Clause, but that theory is squarely foreclosed by the Court’s precedents, which establish that a State’s regulation of abortion is not a sex-based classification and is thus not subject to the heightened scrutiny that applies to such classifications. See Geduldig v. Aiello, 417 U. S. 484, 496, n. 20; Bray v. Alexandria Women’s Health Clinic, 506 U. S. 263, 273– 274.
          \"\"\"
          """}

In [ ]:
# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[prompt, example],
)

In [ ]:
print(response.choices[0].message.content)

Fourteenth Amendment's Due Process Clause (frequency = 1): The Due Process Clause of the Fourteenth Amendment to the United States Constitution prohibits states from denying any person of life, liberty, or property without due process of law.

Fourteenth Amendment's Equal Protection Clause (frequency = 1): The Equal Protection Clause of the Fourteenth Amendment to the United States Constitution provides that no state shall deny to any person within its jurisdiction the equal protection of the laws.

Geduldig v. Aiello, 417 U. S. 484 (frequency = 1): A Supreme Court case in which the Court held that a California disability insurance program that excluded pregnancy-related disabilities does not violate the Equal Protection Clause of the Fourteenth Amendment.

Bray v. Alexandria Women's Health Clinic, 506 U. S. 263 (frequency = 1): A Supreme Court case in which the Court held that the Freedom of Access to Clinic Entrances Act (FACE) was a valid exercise of Congress's power under the Comme

#### Fine-tuned Model Building & Example

In [ ]:
training_fp = '/content/drive/MyDrive/Colab Notebooks/soci40133-homeworks/data/training.jsonl'

In [ ]:
# load training data
training_file_id = client.files.create(
  file=open(training_fp, "rb"),
  purpose="fine-tune"
)

In [ ]:
# fine-tune model
fine_tuned_model = client.fine_tuning.jobs.create(
  training_file=training_file_id.id,
  model="gpt-3.5-turbo"
)
fine_tuned_model.id

'ftjob-8kMpxpZsm9RkhsVUc8eprAx6'

In [ ]:
fine_tuned_model = 'ft:gpt-3.5-turbo-0613:personal::8pkK3HKz'

#### Comparing the Models

In [ ]:
example = {"role": "user", "content": f"""
          Please pull the legislative entities, frequencies, and descriptions from this extract of a SCOTUS opinion:
          \"\"\"
          {text}
          \"\"\"
          """}

In [ ]:
# Vanilla model
response_v = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[prompt, example],
)

In [ ]:
# Fine-tuned model
response_ft = client.chat.completions.create(
    model=fine_tuned_model,
    messages=[prompt, example],
)

In [ ]:
print(response_v.choices[0].message.content)

Mississippi Gestational Age Act (frequency=1): Also referred to as 'the Act.' A ruling that prohibits abortions after a fetus's gestational age is more than fifteen weeks, with exceptions for medical emergencies or "a severe fetal abnormality."
Miss. Code Ann. §41–41–191 (frequency=1): The specific section of the Mississippi Code that contains the language of the Gestational Age Act.
Federal District Court (frequency=1): The district courts are the general trial courts of the federal court system. Each district court has at least one United States District Judge, appointed by the President and confirmed by the Senate for a life term.
Roe v. Wade, 410 U.S. 113 (frequency=1): A landmark Supreme Court decision that ruled that the Constitution protects a woman's right to an abortion. The court's decision recognized that the right to liberty in the Constitution includes the right to decide whether to continue a pregnancy.
Planned Parenthood of Southeastern Pa. v. Casey, 505 U.S. 833 (freque

In [ ]:
print(response_ft.choices[0].message.content)

"""
Mississippi Gestational Age Act (frequency=1): Also referred to as 'the Act.' A ruling that prohibits abortions after a fetus's gestational age is more than fifteen (15) weeks. The law also has narrow exceptions for medical emergencies or "a severe fetal abnormality".
Miss. Code Ann. §41–41–191 (frequency=1): The section of the Mississippi Code that includes the provisions of the Mississippi Gestational Age Act. It specifies that a person shall not intentionally or knowingly perform or induce an abortion of an unborn human being if the probable gestational age is greater than fifteen (15) weeks.
Federal District Court (frequency=1): The district courts are the general trial courts of the federal court system. Each district court has at least one United States District Judge, appointed by the President and confirmed by the Senate for a life term.
Roe v. Wade, 410 U.S. 113 (frequency=1): A landmark Supreme Court decision that ruled that the Constitution protects a woman's right to an

### Reflection
- The fined-tuned model is slightly faster
- funnily enough, the vanilla model performed better than the fine-tuned model, which could be a result of the training examples I provided.

## <font color="blue">*Exercise 3*</font>
<font color="blue">Use LLM to generate some data and compare the differences between model-generated data and actual data. This exercise should not be a repetition of exercise 1. You should focus more on analyzing language nuances, qualitatively or quantitatively. You should also notice how the choice of LLM has possibly impacted the language it uses.

### Generating Summaries of Congressional Legislation
Almost every congressional legislation contains corresponding summaries. Lets try to simulate this process using an LLM and compare the generated to the actual summary.

### Build Model

#### Run Example Model
I have to use gpt-3.5-turbo-0125 because of context window limitations with gpt-3.5-turbo.

In [ ]:
prompt ={"role": "system", "content": """
            You are a summarization assistant. Your job is to summarize U.S. legislation for a general audience.
            """}
example = {"role": "user", "content": f"""
          Please summarize this syllabus of a piece of legislation:
          \"\"\"
          {dobbs_trunc_syllabus}
          \"\"\"
          """}

In [ ]:
# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[prompt, example],
)

In [ ]:
response.choices[0].message.content

"Mississippi's Gestational Age Act prohibits abortions of unborn human beings if their probable gestational age is greater than 15 weeks, except in cases of a medical emergency or severe fetal abnormality. The Act was challenged in court by the Jackson Women’s Health Organization and a doctor, arguing that it violated constitutional rights to abortion established in prior Supreme Court cases, including Roe v. Wade and Planned Parenthood v. Casey. The Supreme Court ultimately ruled that there is no constitutional right to abortion, overturning Roe and Casey and returning the authority to regulate abortion to the states and their elected representatives. The decision emphasized that the right to obtain an abortion is not deeply rooted in the country's history and tradition and can be subject to state regulations under rational-basis review. The Court also considered the workability and effect on other legal doctrines in its decision to overrule Roe and Casey and highlighted the moral que

#### Running against Congressional Legislation

In [ ]:
example_legislation = congress_df.loc[~congress_df.loc[:, 'cleaned_summary'].isna(), ['legislation number', 'title', 'congress', 'cleaned_text', 'cleaned_summary']].loc[0,:]
print(example_legislation.title)
print(example_legislation['legislation number'])
print(example_legislation.cleaned_summary)

Let Doctors Provide Reproductive Health Care Act
H.R. 2907
Let Doctors Provide Reproductive Health Care Act This bill sets out protections for and establishes programs to support health care providers (and those who assist such providers) who offer reproductive health care services that are lawful in the state where the services are provided. Reproductive health care services refer to abortion services; contraceptive services; in vitro fertilization; or other reproductive care, education, and counseling that is provided at a health care site or via telehealth, and in a medically accurate manner. The bill prohibits individuals, entities, and states from preventing, restricting, or otherwise interfering with the provision of lawful reproductive health care services by health care providers. The Department of Justice, individuals, or providers may bring a lawsuit to enforce this bill, and states are not immune from suits for violations. Furthermore, states may not use federal funds to pur

In [ ]:
example = {"role": "user", "content": f"""
          Please summarize this syllabus of a piece of legislation:
          \"\"\"
          {example_legislation.cleaned_text}
          \"\"\"
          """}

In [ ]:
# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[prompt, example],
)

In [ ]:
response.choices[0].message.content

'This is a summary of the Let Doctors Provide Reproductive Health Care Act introduced in the 118th Congress. The Act aims to ensure the right to provide reproductive health care services. It includes provisions such as defining reproductive health care services, prohibiting interference with the provision of such services, outlining enforcement mechanisms, prohibiting the use of federal funds for legal actions against providers of these services, establishing a legal services defense fund, providing security grants for health care providers, ensuring fair liability insurance for providers offering reproductive health care services, and including a severability clause.'

### Altering summaries for a general audience
We can also use a model to convert a legislative summary into one that is understandable by a wider audience.

In [ ]:
prompt ={"role": "system", "content": """
            You are a context rewriting assistant. Your job is to rewrite US congressional legislation summaries for a given context.
            """}
example = {"role": "user", "content": f"""
          Please rewrite this summary for a general audience:
          \"\"\"
          {example_legislation.cleaned_summary}
          \"\"\"
          """}

In [ ]:
# Make the API call
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[prompt, example],
)

In [ ]:
response.choices[0].message.content

'The Let Doctors Provide Reproductive Health Care Act aims to protect and support health care providers offering various reproductive health services like abortion, contraception, and in vitro fertilization. It prevents any interference or restrictions on these lawful services and allows for legal action to enforce these protections. The bill also prevents states from using federal funds to hinder providers or individuals involved in reproductive health care services. Additionally, it provides grants to help health care providers facing legal challenges related to offering reproductive health care services.'

### Reflection
The model is very good at generating summaries and also enables the opportunity to change the audience context of summarizations, which is useful for populations that are unfamiliar with loaded legislative vocabulary.

## <font color="blue">*Exercise 4*</font>

<font color="blue">Compare how LLMs change their performance with different shots on your task. If the evalution criterion is quantifiable, such as classification with ground truth labels, plot and show how accuracy changes. If the evalution criterion cannot be easily quantified, such as the clarity of explaining a concept, use your imagination to do some comparsion (for exmaple, you can ask another LLM to rate its peer :)) If you find close-sourced LLM APIs pricy and are unsatisfied with responses from small-sized open-sourced LLMs, you can try large-sized LLMs (such as 70B version Llama-2) with Petals (see [here](https://colab.research.google.com/drive/1uCphNY7gfAUkdDrTx21dZZwCOUDCMPw8?usp=sharing) and [here](https://colab.research.google.com/drive/1Ervk6HPNS6AYVr3xVdQnY5a-TjjmLCdQ)).

### Identifying Legislation Types
Using LLMs to determine if congressional legislation belongs to one of these categories:
1. Protecting the right to abortion (pro-abortion)
2. Attacking the right to abortion (anti-abortion)
3. Not taking a stance but about abortion (non-stance abortion)
4. Not about abortion but about reproductive healthcare (non-abortion reproductive)
5. Unrelated to reproductive healthcare (unrelated)

### Train-Test Split

In [18]:
# randomly grab a few examples from congressional legislation
exclude_indices = congress_df.loc[congress_df.loc[:, 'cleaned_summary']=='None', :].index
full_set = congress_df.loc[(~congress_df.index.isin(exclude_indices)) & (~congress_df.loc[:, 'cleaned_summary'].isna()) & (congress_df.loc[:, 'cleaned_summary'].str.len() < 1500), ['legislation number', 'title', 'cleaned_summary', 'cleaned_text']].drop_duplicates(subset='title', keep='first').reset_index(drop=True)

In [19]:
full_set.head(5)

,legislation number,title,cleaned_summary,cleaned_text
0,H.R. 4901,Reproductive Health Care Accessibility Act,Reproductive Health Care Accessibility Act Thi...,Congressional Bills 118th Congress From the U....
1,H.R. 1723,Abortion is Health Care Everywhere Act of 2023,Abortion is Health Care Everywhere Act of 2023...,Congressional Bills 118th Congress From the U....
2,H.R. 73,No Pro-Abortion Task Force Act,No Pro Abortion Task Force Act This bill prohi...,Congressional Bills 118th Congress From the U....
3,H.R. 7,No Taxpayer Funding for Abortion and Abortion ...,No Taxpayer Funding for Abortion and Abortion ...,Congressional Bills 118th Congress From the U....
4,H.R. 1297,"To amend title 10, United States Code, to proh...",This bill prohibits the Department of Defense ...,Congressional Bills 118th Congress From the U....


In [20]:
training_indices = [1, 131, 138, 291, 305]

In [21]:
training_examples = full_set.loc[training_indices]
training_examples

,legislation number,title,cleaned_summary,cleaned_text
1,H.R. 1723,Abortion is Health Care Everywhere Act of 2023,Abortion is Health Care Everywhere Act of 2023...,Congressional Bills 118th Congress From the U....
131,H.R. 555,Protecting the Dignity of Unborn Children Act ...,Protecting the Dignity of Unborn Children Act ...,Congressional Bills 117th Congress From the U....
138,H.R. 500,"To prohibit the rescission, suspension, or rev...",This bill prohibits the Department of Health a...,Congressional Bills 117th Congress From the U....
291,H.R. 5795,Unintended Pregnancy Reduction Act of 2006,Unintended Pregnancy Reduction Act of 2006 Ame...,Congressional Bills 109th Congress From the U....
305,S. 2242,Anticounterfeiting Act of 2004,Anticounterfeiting Act of 2004 Amends the Fede...,Congressional Bills 108th Congress From the U....


In [22]:
test_indices = [ 12,  22,  42,  62, 109, 122, 153, 154, 199, 209, 260, 276, 278,290, 298, 316, 323, 365, 380, 387]

In [23]:
test_set = full_set.loc[~full_set.index.isin(training_examples.index), :].reset_index(drop=True)
test_examples = full_set.loc[test_indices]
test_examples.head()

,legislation number,title,cleaned_summary,cleaned_text
12,H.R. 26,Born-Alive Abortion Survivors Protection Act,Born Alive Abortion Survivors Protection Act T...,Congressional Bills 118th Congress From the U....
22,S. 1610,Protecting Service Members and Military Famili...,Protecting Service Members and Military Famili...,Congressional Bills 118th Congress From the U....
42,H.R. 1074,Women’s Public Health and Safety Act,Women's Public Health and Safety Act This bill...,Congressional Bills 118th Congress From the U....
62,H.R. 1670,Abortion is Health Care Everywhere Act of 2021,Abortion is Health Care Everywhere Act of 2021...,Congressional Bills 117th Congress From the U....
109,H.R. 627,Child Custody Protection Act of 2021,Child Custody Protection Act of 2021 This bill...,Congressional Bills 117th Congress From the U....


### Labeling Training Examples

In [24]:
training_examples = training_examples.reset_index(drop=True)
training_examples['legislation_type'] = ['pro-abortion', 'non-stance abortion', 'anti-abortion', 'reproductive healthcare general', 'unrelated']

In [25]:
# join examples
training = []
training_examples.loc[:, ['cleaned_summary', 'legislation_type']].apply(lambda x: training.append(tuple(x)), axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

### Build Model

In [26]:
test_examples = test_examples.reset_index(drop=True)

In [27]:
examples = ""

for i, train in enumerate(training):
  summary, leg_type = train
  examples += f"""
      [Example {i+1}]
      User: Identify the legislation category based on this summary: \"\"\"{summary}\"\"\"
      Response: {leg_type}
      """

print(examples)


      [Example 1]
      User: Identify the legislation category based on this summary: """Abortion is Health Care Everywhere Act of 2023 This bill authorizes using certain foreign assistance funds to provide comprehensive reproductive health care services in developing countries, including abortion services, training, and equipment. The bill also removes a statutory prohibition on using such funds in developing countries to pay for abortions performed as a method of family planning, or motivate or coerce any person to practice abortions."""
      Response: pro-abortion
      
      [Example 2]
      User: Identify the legislation category based on this summary: """Protecting the Dignity of Unborn Children Act of 2021 This bill establishes a new criminal offense for recklessly disposing of or abandoning fetal remains in a landfill or in any navigable waters of the United States. The term fetal remains means any part (except a cremated part) of a deceased human fetus following an aborti

In [28]:
prompt ={"role": "system", "content": f"""
            You are a legislation type identification assistant. Your job is to use congressional legislation summaries delimited with triple quotes to identify legislation into one of four categories:
            1. Protecting the right to abortion (pro-abortion)
            2. Attacking the right to abortion (anti-abortion)
            3. Not taking a stance but about abortion (non-stance abortion)
            4. Not specifically about abortion but about reproductive healthcare in general (reproductive healthcare general)
            5. Unrelated to reproductive healthcare (unrelated)

            {examples}
            """}

In [ ]:
possible_labels = ['pro-abortion', 'anti-abortion', 'non-stance abortion', 'reproductive healthcare general', 'unrelated']
pattern = re.compile('|'.join(map(re.escape, possible_labels)))
labels = []

for i in test_examples.index:
  example = {"role": "user", "content": f"""
          Identify the legislation category based on this summary:
          \"\"\"{test_examples.loc[i, 'cleaned_summary']}\"\"\"
          """}
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[prompt, example],
  )
  label = response.choices[0].message.content
  match = pattern.search(label)
  matched_label = match.group(0) if match else ''

  labels.append(matched_label)

test_examples['predicted_labels'] = labels
test_examples

,legislation number,title,cleaned_summary,cleaned_text,predicted_labels
0,H.R. 26,Born-Alive Abortion Survivors Protection Act,Born Alive Abortion Survivors Protection Act T...,Congressional Bills 118th Congress From the U....,anti-abortion
1,S. 1610,Protecting Service Members and Military Famili...,Protecting Service Members and Military Famili...,Congressional Bills 118th Congress From the U....,reproductive healthcare general
2,H.R. 1074,Women’s Public Health and Safety Act,Women's Public Health and Safety Act This bill...,Congressional Bills 118th Congress From the U....,anti-abortion
3,H.R. 1670,Abortion is Health Care Everywhere Act of 2021,Abortion is Health Care Everywhere Act of 2021...,Congressional Bills 117th Congress From the U....,pro-abortion
4,H.R. 627,Child Custody Protection Act of 2021,Child Custody Protection Act of 2021 This bill...,Congressional Bills 117th Congress From the U....,anti-abortion
5,S. 61,Pain-Capable Unborn Child Protection Act,Pain Capable Unborn Child Protection Act This ...,Congressional Bills 117th Congress From the U....,anti-abortion
6,H.R. 3563,Stop Shackling and Detaining Pregnant Women Act,Stop Shackling and Detaining Pregnant Women Ac...,Congressional Bills 116th Congress From the U....,reproductive healthcare general
7,H.R. 2010,Second Chance at Life Act of 2019,Second Chance at Life Act of 2019 This bill re...,Congressional Bills 116th Congress From the U....,anti-abortion
8,H.R. 2972,Equal Access to Abortion Coverage in Health In...,Equal Access to Abortion Coverage in Health In...,Congressional Bills 114th Congress From the U....,pro-abortion
9,S. 201,Child Custody Protection Act of 2015,Child Custody Protection Act of 2015 Amends th...,Congressional Bills 114th Congress From the U....,anti-abortion


### Evaluate Predictions

#### Using another model
Using another LLM to examine predictions based on legislation text (not summaries)

In [ ]:
prompt ={"role": "system", "content": f"""
            You are a legislation categorization evaluation assistant.
            You will evaluate legislation categorization done by a separate model (categories below).
            The other model's categories were based on legislation summaries.
            You will evaluate the categories based on full legislation text, where text is delimited with triple quotes.
            Your response should provide evaluations by replicating the same label if the label is correct, or providing the corrected label if the label if incorrect.
            Please do not provide extra explainations.

            The potential categories are:
            1. Protecting the right to abortion (pro-abortion)
            2. Attacking the right to abortion (anti-abortion)
            3. Not taking a stance but about abortion (non-stance abortion)
            4. Not specifically about abortion but about reproductive healthcare in general (reproductive healthcare general)
            5. Unrelated to reproductive healthcare (unrelated)

            [Example 1]
            Model 1 prediction: pro-abortion
            Legislation text: \"\"\"This is a piece of sample text with no relevance to the task.\"\"\"
            Evaluation: unrelated

            [Example 2]
            Model 1 prediction: anti-abortion
            Legislation text: \"\"\" Abortion should be banned everywhere.\"\"\"
            Evaluation: anti-abortion
            """}

In [ ]:
possible_labels = ['pro-abortion', 'anti-abortion', 'non-stance abortion', 'reproductive healthcare general', 'unrelated']
pattern = re.compile('|'.join(map(re.escape, possible_labels)))
evaluations = []

for i in test_examples.index:
  example = {"role": "user", "content": f"""
          Evaluate the legislation category based on this text:
          Model 1 prediction: {test_examples.loc[i, 'predicted_labels']}
          Legislation text: \"\"\"{test_examples.loc[i, 'cleaned_text']}\"\"\"
          """}
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[prompt, example],
  )
  evaluation = response.choices[0].message.content
  evaluations.append(evaluation)

test_examples['evaluations'] = evaluations
test_examples

,legislation number,title,cleaned_summary,cleaned_text,predicted_labels,evaluations,alignment,manual_labels
0,H.R. 26,Born-Alive Abortion Survivors Protection Act,Born Alive Abortion Survivors Protection Act T...,Congressional Bills 118th Congress From the U....,anti-abortion,anti-abortion,1,non-stance abortion
1,S. 1610,Protecting Service Members and Military Famili...,Protecting Service Members and Military Famili...,Congressional Bills 118th Congress From the U....,reproductive healthcare general,anti-abortion,0,reproductive healthcare general
2,H.R. 1074,Women’s Public Health and Safety Act,Women's Public Health and Safety Act This bill...,Congressional Bills 118th Congress From the U....,anti-abortion,anti-abortion,1,non-stance abortion
3,H.R. 1670,Abortion is Health Care Everywhere Act of 2021,Abortion is Health Care Everywhere Act of 2021...,Congressional Bills 117th Congress From the U....,pro-abortion,anti-abortion,1,pro-abortion
4,H.R. 627,Child Custody Protection Act of 2021,Child Custody Protection Act of 2021 This bill...,Congressional Bills 117th Congress From the U....,anti-abortion,anti-abortion,1,anti-abortion
5,S. 61,Pain-Capable Unborn Child Protection Act,Pain Capable Unborn Child Protection Act This ...,Congressional Bills 117th Congress From the U....,anti-abortion,anti-abortion,0,anti-abortion
6,H.R. 3563,Stop Shackling and Detaining Pregnant Women Act,Stop Shackling and Detaining Pregnant Women Ac...,Congressional Bills 116th Congress From the U....,reproductive healthcare general,anti-abortion,1,reproductive healthcare general
7,H.R. 2010,Second Chance at Life Act of 2019,Second Chance at Life Act of 2019 This bill re...,Congressional Bills 116th Congress From the U....,anti-abortion,anti-abortion,1,non-stance abortion
8,H.R. 2972,Equal Access to Abortion Coverage in Health In...,Equal Access to Abortion Coverage in Health In...,Congressional Bills 114th Congress From the U....,pro-abortion,anti-abortion,1,pro-abortion
9,S. 201,Child Custody Protection Act of 2015,Child Custody Protection Act of 2015 Amends th...,Congressional Bills 114th Congress From the U....,anti-abortion,anti-abortion,1,anti-abortion


In [ ]:
test_examples_c = test_examples.copy()

In [ ]:
test_examples['alignment'] = 1
test_examples.loc[test_examples.loc[:, 'evaluations'].str.contains('Incorrect'), 'alignment'] = 0

In [ ]:
print(f"Model prediction alignment: {(test_examples.loc[:, 'alignment'].sum()/len(test_examples))*100:.2f}%")

Model prediction alignment: 75.00%


#### Comparison to manual coding

In [ ]:
test_examples=test_examples.reset_index(drop=True)

In [ ]:
manual_labels = ['non-stance abortion',
                 'reproductive healthcare general',
                 'anti-abortion',
                 'pro-abortion',
                 'anti-abortion',
                 'anti-abortion',
                 'reproductive healthcare general',
                 'non-stance abortion',
                 'pro-abortion',
                 'anti-abortion',
                 'anti-abortion',
                 'reproductive healthcare general',
                 'unrelated',
                 'reproductive healthcare general',
                 'anti-abortion',
                 'non-stance abortion',
                 'anti-abortion',
                 'unrelated',
                 'reproductive healthcare general',
                 'non-stance abortion']

In [ ]:
test_examples.loc[:, 'manual_labels'] = manual_labels

In [ ]:
test_examples

,legislation number,title,cleaned_summary,cleaned_text,predicted_labels,evaluations,alignment,manual_labels
0,H.R. 26,Born-Alive Abortion Survivors Protection Act,Born Alive Abortion Survivors Protection Act T...,Congressional Bills 118th Congress From the U....,anti-abortion,"Correct, anti-abortion.",1,non-stance abortion
1,S. 1610,Protecting Service Members and Military Famili...,Protecting Service Members and Military Famili...,Congressional Bills 118th Congress From the U....,reproductive healthcare general,Incorrect; correct label: pro-abortion,0,reproductive healthcare general
2,H.R. 1074,Women’s Public Health and Safety Act,Women's Public Health and Safety Act This bill...,Congressional Bills 118th Congress From the U....,anti-abortion,"Correct, anti-abortion.",1,non-stance abortion
3,H.R. 1670,Abortion is Health Care Everywhere Act of 2021,Abortion is Health Care Everywhere Act of 2021...,Congressional Bills 117th Congress From the U....,pro-abortion,Correct; pro-abortion.,1,pro-abortion
4,H.R. 627,Child Custody Protection Act of 2021,Child Custody Protection Act of 2021 This bill...,Congressional Bills 117th Congress From the U....,anti-abortion,Correct; anti-abortion.,1,anti-abortion
5,S. 61,Pain-Capable Unborn Child Protection Act,Pain Capable Unborn Child Protection Act This ...,Congressional Bills 117th Congress From the U....,anti-abortion,Incorrect; correct label: anti-abortion.,0,anti-abortion
6,H.R. 3563,Stop Shackling and Detaining Pregnant Women Act,Stop Shackling and Detaining Pregnant Women Ac...,Congressional Bills 116th Congress From the U....,reproductive healthcare general,"Correct, reproductive healthcare general.",1,reproductive healthcare general
7,H.R. 2010,Second Chance at Life Act of 2019,Second Chance at Life Act of 2019 This bill re...,Congressional Bills 116th Congress From the U....,anti-abortion,"Correct, anti-abortion.",1,non-stance abortion
8,H.R. 2972,Equal Access to Abortion Coverage in Health In...,Equal Access to Abortion Coverage in Health In...,Congressional Bills 114th Congress From the U....,pro-abortion,Correct; pro-abortion,1,pro-abortion
9,S. 201,Child Custody Protection Act of 2015,Child Custody Protection Act of 2015 Amends th...,Congressional Bills 114th Congress From the U....,anti-abortion,"Correct, anti-abortion.",1,anti-abortion


### Reflection
The model used to estimate the legislation categories based on summaries and the model used to evaluate those estimations based on the full text have pretty good alignment to one another (approximately 75% based on 20 testing legislations). Howver, there is a descrepancy between the two models and my own manual coding, particularly for legislation that mentions abortion without declaring a particular stance on it.

Among those legislations that the two models agreed on in their categorization (n=15), there were 4 discrepancies with my own manual coding, resulting in a 73% alignment. This is not bad, however considering that all mis-alignments happened when categorizing legislation as either anti-abortion or non-stance abortion related, this is a large discrepancy. One way that this could be addressed is by using a fine-tuned model rather than a model with examples.

## <font color="blue">*Exercise 5*</font>

<font color="blue">Using Actor - Critic method to improve LLM's performance on your task or doing some experiments langauge style learning (For example, you can investigate how LLMs perceive different groups of people would write their dating profiles. This may serve as an opportunity to explore how LLMs semantically embed social groups and assess their appropriateness.)

Goal: have an LLM rewrite a pro-abortion legislation to be anti-abortion, and an anti-abortion legislation to be pro-abortion. Have the model from the previous exercise evaluate the newly created legislation.

### Grab Data

In [11]:
pro_legislation = list(congress_df.loc[congress_df.loc[:, 'title'] == 'Abortion is Health Care Everywhere Act of 2021', 'cleaned_text'])[0]

In [17]:
anti_legislation = list(congress_df.loc[congress_df.loc[:, 'title']=="Taxpayers' Freedom of Conscience Act of 2011", 'cleaned_text'])[0]

### Build models

In [39]:
eval_prompt ={"role": "system", "content": f"""
            You are a legislation type identification assistant. Your job is to use congressional legislation summaries delimited with triple quotes to identify legislation into one of four categories:
            1. Protecting the right to abortion (pro-abortion)
            2. Attacking the right to abortion (anti-abortion)
            3. Not taking a stance but about abortion (non-stance abortion)
            4. Not specifically about abortion but about reproductive healthcare in general (reproductive healthcare general)
            5. Unrelated to reproductive healthcare (unrelated)

            {examples}
            """}

In [29]:
anti_to_pro_prompt={"role": "system", "content": f"""
            You are a legislation rewrite assistant. Your goal is to rewrite a piece of legislation delimited by triple quotes to be in favor of abortion access.
            """}

pro_to_anti_prompt={"role": "system", "content": f"""
            You are a legislation rewrite assistant. Your goal is to rewrite a piece of legislation delimited by triple quotes to be opposed to abortion access.
            """}

In [31]:
anti_example = {"role": "user", "content": f"""
          Rewrite this congressional legislation to be in favor of abortion:
          \"\"\"{anti_legislation}\"\"\"
          """}

pro_example = {"role": "user", "content": f"""
          Rewrite this congressional legislation to be opposed to abortion:
          \"\"\"{pro_legislation}\"\"\"
          """}

In [32]:
anti_to_pro_resp = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[anti_to_pro_prompt, anti_example],
  )

pro_to_anti_resp = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[pro_to_anti_prompt, pro_example],
  )

### Evaluations

#### Pro to Anti

In [36]:
anti_legislation

"Congressional Bills 112th Congress From the U.S. Government Publishing Office H.R. 1099 Introduced in House (IH) 112th CONGRESS 1st Session H. R. 1099 To prohibit any Federal official from expending any Federal funds for any population control or population planning program or any family planning activity. IN THE HOUSE OF REPRESENTATIVES March 15, 2011 Mr. Paul introduced the following bill; which was referred to the Committee on Foreign Affairs, and in addition to the Committee on Energy and Commerce, for a period to be subsequently determined by the Speaker, in each case for consideration of such provisions as fall within the jurisdiction of the committee concerned A BILL To prohibit any Federal official from expending any Federal funds for any population control or population planning program or any family planning activity. Be it enacted by the Senate and House of Representatives of the United States of America in Congress assembled, SECTION 1. SHORT TITLE. This Act may be cited a

In [37]:
anti_to_pro_leg = anti_to_pro_resp.choices[0].message.content
anti_to_pro_leg

'"""Congressional Bills 112th Congress From the U.S. Government Publishing Office H.R. 1099 Introduced in House (IH) 112th CONGRESS 1st Session H. R. 1099 To ensure access to reproductive healthcare services for all individuals. IN THE HOUSE OF REPRESENTATIVES March 15, 2011 Mr. Paul introduced the following bill; which was referred to the Committee on Foreign Affairs, and in addition to the Committee on Energy and Commerce, for a period to be subsequently determined by the Speaker, in each case for consideration of such provisions as fall within the jurisdiction of the committee concerned A BILL To ensure the availability of Federal funds for family planning activities, including abortion procedures, to support individuals\' reproductive health choices. Be it enacted by the Senate and House of Representatives of the United States of America in Congress assembled, SECTION 1. SHORT TITLE. This Act may be cited as the Reproductive Healthcare Access Act of 2011\'\'. SEC. 2. ENSURING FEDER

In [40]:
example = {"role": "user", "content": f"""
          Identify the legislation category based on this summary:
          \"\"\"{anti_to_pro_leg}\"\"\"
          """}

eval_resp = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[eval_prompt, example],
  )

In [41]:
eval_resp.choices[0].message.content

'pro-abortion'

#### Anti to Pro

In [42]:
pro_legislation

"Congressional Bills 117th Congress From the U.S. Government Publishing Office H.R. 1670 Introduced in House (IH) 117th CONGRESS 1st Session H. R. 1670 To amend the Foreign Assistance Act of 1961 to authorize the use of funds for comprehensive reproductive health care services, and for other purposes. IN THE HOUSE OF REPRESENTATIVES March 9, 2021 Ms. Schakowsky (for herself, Ms. Adams, Mr. Aguilar, Mr. Auchincloss, Ms. Barragan, Ms. Bass, Mr. Bera, Mr. Beyer, Mr. Blumenauer, Ms. Blunt Rochester, Ms. Bonamici, Mr. Bowman, Mr. Brown, Ms. Brownley, Ms. Bush, Mr. Carbajal, Mr. Cardenas, Mr. Carson, Mr. Cartwright, Mr. Case, Mr. Casten, Ms. Castor of Florida, Mr. Castro of Texas, Ms. Chu, Mr. Cicilline, Ms. Clark of Massachusetts, Ms. Clarke of New York, Mr. Cohen, Mr. Connolly, Mr. Crist, Mr. Crow, Mr. Danny K. Davis of Illinois, Ms. Dean, Mr. DeFazio, Ms. DeLauro, Ms. DelBene, Mr. Deutch, Mr. Doggett, Ms. Escobar, Ms. Eshoo, Mr. Espaillat, Mrs. Fletcher, Mr. Foster, Ms. Lois Frankel of Fl

In [43]:
pro_to_anti_leg = pro_to_anti_resp.choices[0].message.content
pro_to_anti_leg

'"""Congressional Bills 117th Congress From the U.S. Government Publishing Office H.R. 1670 Introduced in House (IH) 117th CONGRESS 1st Session H. R. 1670 To amend the Foreign Assistance Act of 1961 to prohibit the use of funds for comprehensive reproductive health care services, and for other purposes. IN THE HOUSE OF REPRESENTATIVES March 9, 2021 Ms. Schakowsky (for herself, Ms. Adams, Mr. Aguilar, Mr. Auchincloss, Ms. Barragan, Ms. Bass, Mr. Bera, Mr. Beyer, Mr. Blumenauer, Ms. Blunt Rochester, Ms. Bonamici, Mr. Bowman, Mr. Brown, Ms. Brownley, Ms. Bush, Mr. Carbajal, Mr. Cardenas, Mr. Carson, Mr. Cartwright, Mr. Case, Mr. Casten, Ms. Castor of Florida, Mr. Castro of Texas, Ms. Chu, Mr. Cicilline, Ms. Clark of Massachusetts, Ms. Clarke of New York, Mr. Cohen, Mr. Connolly, Mr. Crist, Mr. Crow, Mr. Danny K. Davis of Illinois, Ms. Dean, Mr. DeFazio, Ms. DeLauro, Ms. DelBene, Mr. Deutch, Mr. Doggett, Ms. Escobar, Ms. Eshoo, Mr. Espaillat, Mrs. Fletcher, Mr. Foster, Ms. Lois Frankel of 

In [44]:
example = {"role": "user", "content": f"""
          Identify the legislation category based on this summary:
          \"\"\"{pro_to_anti_leg}\"\"\"
          """}

eval_resp = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[eval_prompt, example],
  )

In [45]:
eval_resp.choices[0].message.content

'pro-abortion'

### Reflection
The quality of the generated legislaiton definitely depends on the input and could benefit from training examples. The legislation created by converted the anti-abortion bill to a pro-abortion bill read as an actual piece of congressional legislation with no problem. However, the legislation created by converting the pro-abortion bill to an anti-abortion one simply added "not" or "no" to many of the original sentences, which technically did change the type of legislation but also resulted in silly assertions where the evaluation model still clocked it as pro-abortion.